## 七、问题与练习

### 1. 问题
#### 【问题一】 如何更改列或行的顺序？如何交换奇偶行（列）的顺序？
#### 【问题二】 如果要选出DataFrame的某个子集，请给出尽可能多的方法实现。
#### 【问题三】 query函数比其他索引方法的速度更慢吗？在什么场合使用什么索引最高效？
#### 【问题四】 单级索引能使用Slice对象吗？能的话怎么使用，请给出一个例子。
#### 【问题五】 如何快速找出某一列的缺失值所在索引？
#### 【问题六】 索引设定中的所有方法分别适用于哪些场合？怎么直接把某个DataFrame的索引换成任意给定同长度的索引？
#### 【问题七】 多级索引有什么适用场合？
#### 【问题八】 什么时候需要重复元素处理？

In [92]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv',index_col='ID').sort_index()
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [93]:
#【问题一】 如何更改列或行的顺序？如何交换奇偶行（列）的顺序？
#提出index和columns
idxlist = (df.index.values)
collist = (df.columns.values)
#调整index和columns位置
i = 0;j = 3
idxlist[i],idxlist[j] = idxlist[j],idxlist[i]
collist = collist[0::2]
#使用df.reindex()。修改原始df的。
df.reindex(index = idxlist, columns = collist).head()

,School,Gender,Height,Math
ID,,,,
1104,S_1,M,173,34.0
1102,S_1,F,192,32.5
1103,S_1,M,186,87.2
1101,S_1,F,167,80.4
1105,S_1,F,159,84.8


In [94]:
#【问题二】 如果要选出DataFrame的某个子集，请给出尽可能多的方法实现。
%timeit df.loc[df['Height']>165,'Height':'Math']#全闭
df.iloc[1:15:3,4:7]#半闭半开
%timeit df[df['Height']>165][['Height','Weight','Math']]
df.query('(ID in [1102,1105,1203,1301,1304])')[['Height','Weight','Math']]

700 µs ± 8.61 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
971 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


,Height,Weight,Math
ID,,,
1102,192,73,32.5
1105,159,64,84.8
1203,160,53,58.8
1301,161,68,31.5
1304,195,70,85.2


In [95]:
## 【问题三】 query函数比其他索引方法的速度更慢吗？在什么场合使用什么索引最高效？
%timeit df.loc[1102:2401:3,'Height':'Math']
%timeit df.query('(Address in ["street_6","street_7"])&(Weight>(70+10))&(ID in [1303,2304,2402])')
#慢很多，
'''
1、一般来说，[]操作符常用于列选择或布尔选择，筛条目，操作只能输入一个维度。尽量避免行的index选择。优点表达简单。
2、逻辑较简单的索引使用iloc/loc，可以调用函数，并同时控制两个维度。loc可看作[]的扩充：
    避免chained assignment
    在对数据赋值使可以有以下两种操作：
        df.loc[df.C>0.5,'C']=0.6
        df[df.C>0.5][['C']]=0.6
    上面的操作可以在df上直接修改数据，而下面的操作被称为chained assignment,会产生SettingWithCopyWarning，官网上的解释：indexing-view-versus-copy。
    直观来说就使用chained assignment无法预测是否能对数据df进行修改，使用loc一定是直接在df上修改数据，因此建议使用loc
    
3、数据量较大时复杂索引时使用query，bool表达式。
eval/query用在数据（df.values.nbytes>1万）性能提升明显；传统方法在小数组时运行得更快
'''


416 µs ± 4.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.37 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


"\n1、一般来说，[]操作符常用于列选择或布尔选择，筛条目，操作只能输入一个维度。尽量避免行的index选择。优点表达简单。\n2、逻辑较简单的索引使用iloc/loc，可以调用函数，并同时控制两个维度。loc可看作[]的扩充：\n    避免chained assignment\n    在对数据赋值使可以有以下两种操作：\n        df.loc[df.C>0.5,'C']=0.6\n        df[df.C>0.5][['C']]=0.6\n    上面的操作可以在df上直接修改数据，而下面的操作被称为chained assignment,会产生SettingWithCopyWarning，官网上的解释：indexing-view-versus-copy。\n    直观来说就使用chained assignment无法预测是否能对数据df进行修改，使用loc一定是直接在df上修改数据，因此建议使用loc\n    \n3、数据量较大时复杂索引时使用query，bool表达式。\neval/query用在数据（df.values.nbytes>1万）性能提升明显；传统方法在小数组时运行得更快\n"

In [96]:
#【问题四】 单级索引能使用Slice对象吗？能的话怎么使用，请给出一个例子。
#可以使用。
df_s = pd.DataFrame(np.random.rand(9,9))
idx=pd.IndexSlice
df_s.loc[idx[3:6],idx[df_s.sum()>4]]

,0,3,4,5,6,8
3,0.448650,0.094556,0.059766,0.674296,0.666517,0.620790
4,0.516195,0.751881,0.890548,0.147978,0.709756,0.093256
5,0.639662,0.360989,0.521474,0.486887,0.754230,0.846486
6,0.935597,0.345319,0.470610,0.693860,0.412275,0.997469


In [97]:
#【问题五】 如何快速找出某一列的缺失值所在索引？
df_f = pd.DataFrame({'A':[10,2,3,100,-1]},index=[1,2,6,5,7])
df_f['B']=list('abbc1')
df_f = df_f.assign(C=pd.Series(list('deffg')))#Seires默认index从0开始，错位
print(df_f)

df_f[df_f['C'].isna()].index

     A  B    C
1   10  a    e
2    2  b    f
6    3  b  NaN
5  100  c  NaN
7   -1  1  NaN


Int64Index([6, 5, 7], dtype='int64')

In [128]:
#【问题六】 索引设定中的所有方法分别适用于哪些场合？怎么直接把某个DataFrame的索引换成任意给定同长度的索引？
#index_col    ：index_col是read_csv中的一个参数，而不是某一个方法。read新表时设定
pd.read_csv('data/table.csv',index_col=['Address','School']).head()

#包含着索引对齐的逻辑
#reindex      ：reindex是指重新索引，它的重要特性在于索引对齐，很多时候用于索引间的重新排序。
df = pd.read_csv('data/table.csv',index_col='ID').sort_index()
df.reindex(index=[1101,1203,1204,1206,2402,1205],method='bfill')#填充方法method仅适用于已排序的index调整。填充在【原始】条件下执行
df.sort_index().reindex(columns=['Average','Height','Gender']).head()#调整column不能填补
#reindex_like ：生成一个横纵索引完全与参数列表一致的DataFrame，数据使用被调用的表
df_temp = pd.DataFrame({'Weight':range(5),
                        'Height':range(5),
                        'ID':[1101,1104,1103,1106,1102]}).set_index('ID').sort_index()
print('原始',df_temp)
df_temp.reindex_like(df[0:5][['Weight','Height']],method='bfill')#原表index有序，这里的1105的值由bfill规则参照1106填充

#Series级名称修改index
#set_index    ：将某些列作为索引，使用表内列、与表长相同的Series列，支持直接添加多级索引。append=True保留原index。
#reset_index  ：默认恢复自然数索引，用level参数指定哪一层index被reset，用col_level参数指定set到column的哪一层。

#元素级名称修改，不改数据
#rename_axis  ：针对多级索引的方法，作用是修改某一层的索引名，而不是索引标签。修改坐标轴名。
#rename       ：修改列或者行索引标签，而不是索引名。修改坐标轴上的点名称。

原始       Weight  Height
ID                  
1101       0       0
1102       4       4
1103       2       2
1104       1       1
1106       3       3


,Weight,Height
ID,,
1101,0,0
1102,4,4
1103,2,2
1104,1,1
1105,3,3


In [ ]:
#【问题七】 多级索引有什么适用场合？
# 1、处理多级结构数据时，例如学生信息，年级-班级。索引较清晰。
# 2、将某些离散feature转为多级索引，在结构上突出重要性。同时联合索引Slice可以灵活查询


In [129]:
#【问题八】 什么时候需要重复元素处理？
# 1、清洗数据时去掉多余重复数据；
# 2、统计元素种类时
# 3、透视表中交叉查询时

### 2. 练习
#### 【练习一】 现有一份关于UFO的数据集，请解决下列问题：

In [350]:
df_ufo = pd.read_csv('data/UFO.csv')
df_ufo.head()

,datetime,shape,duration (seconds),latitude,longitude
0,10/10/1949 20:30,cylinder,2700.0,29.883056,-97.941111
1,10/10/1949 21:00,light,7200.0,29.384210,-98.581082
2,10/10/1955 17:00,circle,20.0,53.200000,-2.916667
3,10/10/1956 21:00,circle,20.0,28.978333,-96.645833
4,10/10/1960 20:00,light,900.0,21.418056,-157.803611


#### （a）在所有被观测时间超过60s的时间中，哪个形状最多？

In [358]:
df_ufo[df_ufo['duration (seconds)']>60]['shape'].value_counts().index[0]

'light'

#### （b）对经纬度进行划分：-180°至180°以30°为一个划分，-90°至90°以18°为一个划分，请问哪个区域中报告的UFO事件数量最多？

In [383]:
lon_interval = pd.cut(df_ufo['longitude'],bins=list(range(-180,181,30)))
lat_interval = pd.cut(df_ufo['latitude'],bins=list(range(-90,91,18)))
df_i = df_ufo.join(lon_interval,rsuffix='_interval').join(lat_interval,rsuffix='_interval')
pd.Series(list(zip(df_i['latitude_interval'],df_i['longitude_interval']))).value_counts().index[0]

(Interval(36, 54, closed='right'), Interval(-90, -60, closed='right'))

#### 【练习二】 现有一份关于口袋妖怪的数据集，请解决下列问题：

In [388]:
df_poke = pd.read_csv('data/Pokemon.csv')
df_poke.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


#### （a）双属性的Pokemon占总体比例的多少？

In [407]:
#Pokemon的Type 1都非空，因此只需判断Type 2是否非空
df_poke[df_poke['Type 2'].notna()].shape[0]/df_poke.shape[0]

0.5175

#### （b）在所有种族值（Total）不小于580的Pokemon中，非神兽（Legendary=False）的比例为多少？


In [412]:
df_580 = df_poke[df_poke['Total']>=580] #提取种族值不小于580
df_580[df_580['Legendary']==False].shape[0]/df_580.shape[0]

0.4247787610619469

#### （c）在第一属性为格斗系（Fighting）的Pokemon中，物攻排名前三高的是哪些？


In [429]:
df_poke[df_poke['Type 1']=='Fighting'].sort_values(by='Attack',ascending=False)['Name'][0:3]

498    LucarioMega Lucario
594             Conkeldurr
74                 Machamp
Name: Name, dtype: object

#### （d）请问六项种族指标（HP、物攻、特攻、物防、特防、速度）极差(每条数据内横向计算max-min)的均值最大的是哪个属性（只考虑第一属性，且均值是对属性而言）？


In [443]:
df_poke['range'] = df_poke.iloc[:,5:11].max(axis=1)-df_poke.iloc[:,5:11].min(axis=1)
#第一句参考答案，用于理解题目：每条数据内六种参数横向计算极差max-min
df_poke.groupby('Type 1').mean().sort_values(by='range',ascending=False).index[0]

'Steel'

#### （e）哪个属性（只考虑第一属性）的神兽比例最高？该属性神兽的种族值(Total)也是最高的吗？

In [495]:
#参考答案，计算的是哪个属性的神兽数量最多
df_poke.query('Legendary == True')['Type 1'].value_counts(normalize=True).index[0]

'Psychic'

In [494]:
#如果理解为，哪个属性里神兽比例更高。最高的是Flying，神兽比例50%
df_poke_all = df_poke.groupby('Type 1').count()['Legendary']
df_poke_True   = df_poke[df_poke['Legendary'] == True].groupby('Type 1').count()['Legendary'].reindex(index=df_poke_all.index)
df_poke_Tp  = df_poke_T.fillna(0)/df_poke_all
df_poke_Tp.sort_values(ascending=False)

Type 1
Flying      0.500000
Dragon      0.375000
Psychic     0.245614
Steel       0.148148
Ground      0.125000
Fire        0.096154
Rock        0.090909
Electric    0.090909
Ice         0.083333
Dark        0.064516
Ghost       0.062500
Fairy       0.058824
Grass       0.042857
Water       0.035714
Normal      0.020408
Fighting    0.000000
Poison      0.000000
Bug         0.000000
Name: Legendary, dtype: float64

In [497]:
#第二问，比较哪个属性的神兽的Total均值最高
df_poke[df_poke['Legendary'] == True].groupby('Type 1').mean()['Total'].sort_values(ascending=False)

Type 1
Normal      695.000000
Ghost       680.000000
Fairy       680.000000
Water       675.000000
Dragon      673.333333
Ground      660.000000
Dark        640.000000
Psychic     638.571429
Rock        615.000000
Steel       610.000000
Fire        608.000000
Grass       593.333333
Ice         580.000000
Flying      580.000000
Electric    580.000000
Name: Total, dtype: float64